In [ ]:
#pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
chromedriver = "C:/Users/anush/Downloads/chromedriver_win32/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.pmindia.gov.in/en/mann-ki-baat/")

ScrollNumber = 30
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollTo(1,50000)")
    time.sleep(5)

In [ ]:
file = open('DS.html', 'w', encoding='utf-8')
file.write(driver.page_source)
file.close()

driver.close()

In [ ]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

In [ ]:
import urllib.request
import requests
from bs4 import BeautifulSoup

with open('DS.html', 'r', encoding='utf-8') as f:
    page = f.read()

parser = BeautifulSoup(page, 'html.parser')

links = []
for i in parser.find_all('div', attrs={'class' : 'mkb-news-wrapper'}):
    links.append(i.a['href'])

len(links)

In [ ]:
for i in range(len(links)) :
    speech_response =  requests.get(links[i])
    speech_content = speech_response.content
#     file = open('All_speech/speech'+str(i)+'.html', 'w', encoding='utf-8')
#     file.write(speech_response.content)
#     file.close()

#     with open('All_speech/speech'+str(i)+'.html', "w", encoding='utf-8') as f:
#       f.write(speech_response.content)
    
    save_html(speech_response.content, 'All_speech/speech'+str(len(links)-i)+'.html')
    file.close()
    
    time.sleep(10)
    speech_parser = BeautifulSoup(speech_content, 'html.parser')

    content = speech_parser.select(".content-block")
    #print(content)
   

In [ ]:
speech_date = []
speech = []

for i in range(len(links)) :
    html = open_html('All_speech/speech'+str(i+1)+'.html')
    parser = BeautifulSoup(html, 'html.parser')
    speech_text = parser.select('.left-container')
    for s in speech_text : 
        date = s.select(".share_date .date")[0].text
        speech_date.append(date)
        speech_ = parser.select(".news-bg")[0].text
        speech.append(speech_)

In [ ]:
# print(speech_date)
# print("\n")
# print(speech)

In [ ]:
import pandas as pd

col1 = speech_date
col2 = speech
mann_ki_baat = pd.DataFrame(
    {'Speech_date': col1,
     'Speech': col2
    })
mann_ki_baat

In [ ]:
cleaned = []
for date in mann_ki_baat['Speech_date']:
    clean = str(date).replace('\\n', '')
    clean = clean.replace("\'", '')
    cleaned.append(clean)
    
mann_ki_baat['Speech_date'] = cleaned

cleaned = []
for sp in mann_ki_baat['Speech']:
    clean = str(sp).replace('\n', '').replace("\'", '')
    clean = clean.replace('(Hon’ble Shri Narendra Modi):', '').replace('\\xa0', ' ')
    cleaned.append(clean)
    
mann_ki_baat['Speech'] = cleaned
mann_ki_baat

In [ ]:
mann_ki_baat.to_csv("mann_ki_baat.csv")

# LDA
---

In [1]:
import pandas as pd
import re
import string
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora

all_speeches = pd.read_csv('mann_ki_baat.csv', encoding='UTF-8')
all_speeches = all_speeches.drop(['Unnamed: 0'], axis= 1)
all_speeches

,Speech_date,Speech
0,"03 Oct, 2014","My Dear Countrymen,Today is the holy festival ..."
1,"02 Nov, 2014","My dear fellow countrymen, I am with you again..."
2,"14 Dec, 2014","My Dear Fellow Countrymen,Today I have this gr..."
3,"27 Jan, 2015","Today, Shri Barack Obama, President of the Uni..."
4,"22 Feb, 2015","Hello, my young friends. Today probably the en..."
...,...,...
68,"29 Nov, 2020","My dear countrymen,Namaskar! I want to share a..."
69,"27 Dec, 2020","My dear countrymen,Namaskar. Today is the 27th..."
70,"31 Jan, 2021","My dear countrymen,Namaskar. When I express Ma..."
71,"28 Feb, 2021","My dear countrymen,Namaskar. Yesterday was the..."


In [ ]:
def preprocess(text):
    input_str = text.lower()
    input_str = re.sub(r'\d+', '', input_str)
    input_str = input_str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    result = input_str.strip()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(result)
    result = [i for i in tokens if not i in stop_words]
    
    remove = ['’','‘','“', '”', 'th', 'nd', 'st','ji','ii']
    final = []
    for word in result :
        if word not in remove :
            final.append(word)
        else :
            pass
    return result

In [ ]:
all_speeches['Preprocessed'] = all_speeches['Speech'].apply(preprocess)

In [ ]:
all_speeches


In [ ]:
# from nltk.stem import WordNetLemmatizer
# # nltk.download('wordnet')
# from nltk.tokenize import word_tokenize
# lemmatizer=WordNetLemmatizer()
# lema_words = []

# for word in result3:
#     lema_words.append(lemmatizer.lemmatize(word))

In [20]:
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

def lemmatization(text, allowed_postags=['PROPN', 'NOUN', 'ADJ']):
    output = []
    for sent in text :
        doc = nlp(sent)
#         output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#         output.append([token for token in doc])
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
from nltk.tokenize import sent_tokenize

tokenized_sent=sent_tokenize(all_speeches['Speech'][0])
print(tokenized_sent)

['My Dear Countrymen,Today is the holy festival of Vijay Dashami.', 'My heartiest greetings on this occasion of Vijay Dashami to one and all.Through the medium of radio, I would like to share few heartfelt thoughts with you today.', 'And, I hope that not only today, this series of conversation may be carried out regularly in future.', 'I will try my best, if possible, to take out time twice a month or even once to speak with you.', 'In future, I have also decided that whenever I will speak to you, it would be on Sundays and time would be morning 11.', 'In that way, it would be convenient for you too and I will feel contented with the fact that I am successful in sharing my thoughts with you.We are celebrating the festival of Vijay Dashami today, which symbolises the triumph of Good over Evil.', 'But one gentleman named Ganesh Venkatadari, a native of Mumbai, sent me a mail and has written to me that we must take a vow to eliminate ten bad habits from within ourselves on this occasion o

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# for i in range(len(all_speeches)) :
tokenized = lemmatization(tokenized_sent)
print(tokenized)

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['Dear', 'Countrymen', 'today', 'holy', 'festival', 'Vijay', 'Dashami'], ['heartiest', 'greeting', 'occasion', 'Vijay', 'Dashami', 'medium', 'radio', 'few', 'heartfelt', 'thought', 'today'], ['today', 'series', 'conversation', 'future'], ['good', 'possible', 'time', 'month'], ['future', 'Sundays', 'time', 'morning'], ['way', 'convenient', 'fact', 'successful', 'thought', 'festival', 'Vijay', 'Dashami', 'today', 'triumph', 'good', 'Evil'], ['gentleman', 'Ganesh', 'Venkatadari', 'native', 'Mumbai', 'mail', 'vow', 'bad', 'habit', 'occasion', 'Vijay', 'Dashami'], ['gratitude', 'recommendation'], ['individual', 'end', 'bad', 'habit', 'respect', 'nation', 'vow', 'dirt', 'country'], ['occasion', 'Vijay', 'Dashami', 'vow', 'dirt', 'filth', 'occasion', 'yesterday', '2nd', 'October', 'eve', 'Mahatma', 'Gandhi', 'birth', 'anniversary', 'more', 'crore', 'countryman', 'Swachh', 'Bharat', 'movement'], ['thought', 'yesterday', 'people', 'video', 'nation', 'social', 'medium', 'website', 'more', 'peop

In [24]:
stop_words = set(stopwords.words('english'))
result = []
for i in range(len(tokenized)) :
    tokens = tokenized[i]
    result.append([i for i in tokens if not i in stop_words])
print(result)

[['Dear', 'Countrymen', 'today', 'holy', 'festival', 'Vijay', 'Dashami'], ['heartiest', 'greeting', 'occasion', 'Vijay', 'Dashami', 'medium', 'radio', 'heartfelt', 'thought', 'today'], ['today', 'series', 'conversation', 'future'], ['good', 'possible', 'time', 'month'], ['future', 'Sundays', 'time', 'morning'], ['way', 'convenient', 'fact', 'successful', 'thought', 'festival', 'Vijay', 'Dashami', 'today', 'triumph', 'good', 'Evil'], ['gentleman', 'Ganesh', 'Venkatadari', 'native', 'Mumbai', 'mail', 'vow', 'bad', 'habit', 'occasion', 'Vijay', 'Dashami'], ['gratitude', 'recommendation'], ['individual', 'end', 'bad', 'habit', 'respect', 'nation', 'vow', 'dirt', 'country'], ['occasion', 'Vijay', 'Dashami', 'vow', 'dirt', 'filth', 'occasion', 'yesterday', '2nd', 'October', 'eve', 'Mahatma', 'Gandhi', 'birth', 'anniversary', 'crore', 'countryman', 'Swachh', 'Bharat', 'movement'], ['thought', 'yesterday', 'people', 'video', 'nation', 'social', 'medium', 'website', 'people'], ['nation', 'peopl

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
dictionary = corpora.Dictionary(tokenized)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized]

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
LDA = gensim.models.ldamodel.LdaModel

lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, 
                random_state=100, chunksize=1, passes=50, iterations=10)

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
lda_model.print_topics()

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.145*"strength" + 0.035*"home" + 0.031*"skill" + 0.030*"medium" + 0.021*"development" + 0.021*"course" + 0.020*"inner" + 0.018*"hut" + 0.018*"village" + 0.018*"Youths"'),
 (1,
  '0.153*"suggestion" + 0.044*"countryman" + 0.037*"people" + 0.033*"crore" + 0.017*"numerous" + 0.017*"advice" + 0.017*"am" + 0.017*"love" + 0.016*"Sundays" + 0.013*"effective"'),
 (2,
  '0.091*"speech" + 0.091*"original" + 0.050*"step" + 0.047*"version" + 0.047*"authoritative" + 0.044*"english" + 0.044*"rendering" + 0.044*"Hindi" + 0.028*"corner" + 0.014*"Dashami"'),
 (3,
  '0.296*"nation" + 0.039*"trust" + 0.022*"Sisters" + 0.022*"poor" + 0.017*"height" + 0.017*"new" + 0.016*"farmer" + 0.016*"village" + 0.016*"Youths" + 0.016*"hut"'),
 (4,
  '0.125*"thought" + 0.049*"today" + 0.047*"time" + 0.035*"heartfelt" + 0.030*"journey" + 0.027*"heart" + 0.023*"fortunate" + 0.023*"beginning" + 0.019*"incident" + 0.018*"future"')]

In [54]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
vis

C:\Users\anush\anaconda3\envs\nlp\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.162962 -0.060841       1        1  26.410604
1     -0.111871 -0.008952       2        1  26.011924
2      0.032558  0.215845       3        1  17.688874
3      0.144174 -0.078480       4        1  16.164470
0      0.098100 -0.067573       5        1  13.724128, topic_info=           Term       Freq      Total Category  logprob  loglift
45       nation  26.000000  26.000000  Default  30.0000  30.0000
187  suggestion  21.000000  21.000000  Default  29.0000  29.0000
14      thought  18.000000  18.000000  Default  28.0000  28.0000
110    strength  11.000000  11.000000  Default  27.0000  27.0000
288      speech   9.000000   9.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
120     Sisters   0.903267   3.293436   Topic5  -4.3942   0.6923
275       Radio   0.751389   3.087306   Topic5  -4.5783   0.5729
276        glad   0.751389   3.087306   Topic5  -4.5783   0.5729
16       future   1.128695   5.022377   Topic5  -4.1714   0.4932
277      simple   0.751389   3.087306   Topic5  -4.5783   0.5729

[206 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
235       5  0.740844         5th
1         3  0.423641     Dashami
225       4  0.523789  Government
185       2  0.670518     Gujarat
284       3  0.840687       Hindi
...     ...       ...         ...
244       5  0.711340     various
290       3  0.782219     version
283       4  0.307556     village
283       5  0.307556     village
29        1  0.582350         way

[151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])

# Sentiment Analysis
---